In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
import psycopg2
from sqlalchemy import create_engine

today = datetime.date.today()
yesterday = datetime.date.today() - datetime.timedelta(days=1)
url = 'https://www.imdb.com/search/title/?title_type=feature&year='+ str(yesterday) +','+ str(today) +'&start=1&ref_=adv_nxt'

titles = []
years = []
ratings = []
genres = []
runtimes = []
imdb_ratings = []
metascores = []
votes = []

page = requests.get(url)

soup = BeautifulSoup(page.text,  "html.parser")
movie_box = soup.find_all('div', class_ = 'lister-item mode-advanced')
for box in movie_box:

    if box.find('h3', class_ = 'lister-item-header') is not None:
        title = box.find('h3', class_ = 'lister-item-header').a.text
        titles.append(title)
    else:
        titles.append('None')


    if box.h3.find('span', class_= 'lister-item-year text-muted unbold') is not None: 
            #year released
        year = box.h3.find('span', class_= 'lister-item-year text-muted unbold').text # remove the parentheses around the year and make it an integer
        years.append(year)
    else:
        years.append(None)

    if box.p.find('span', class_ = 'certificate') is not None:
        #rating
        rating = box.p.find('span', class_= 'certificate').text
        ratings.append(rating)
    else:
        ratings.append("No rating")

    if box.p.find('span', class_ = 'genre') is not None:  
            #genre
        genre = box.p.find('span', class_ = 'genre').text.replace("\n", "").rstrip().split(',') # remove the whitespace character, strip, and split to create an array of genres
        genres.append(genre)  
    else:
        genres.append("No genre")

    if box.p.find('span', class_ = 'runtime') is not None:
            #runtime
        time = int(box.p.find('span', class_ = 'runtime').text.replace(" min", "")) # remove the minute word from the runtime and make it an integer
        runtimes.append(time)
    else:
        runtimes.append(None)

    if box.find('div', class_ = 'inline-block ratings-imdb-rating') is not None:
        #IMDB ratings
        imdb = float(box.find('div', class_ = 'inline-block ratings-imdb-rating').text) # non-standardized variable
        imdb_ratings.append(imdb)
    else:
        imdb_ratings.append(None)

    if box.find('span', class_ = 'metascore') is not None:
            #Metascore
        m_score = int(box.find('span', class_ = 'metascore').text) # make it an integer
        metascores.append(m_score)
    else:
        metascores.append(None)

    if box.find('p', class_ = 'sort-num_votes-visible') is not None:
            #Number of votes
        vote = int(box.find('p', class_ = 'sort-num_votes-visible').text.replace('\n','').replace('Votes:','').replace(',',''))
        votes.append(vote)
    else:
        votes.append(None)
print('ended')

        

movie_df = pd.DataFrame({'movie': titles, 
                        'year': years,
                        'rating': ratings,
                        'genre': genres,
                        'runtime_min': runtimes,
                        'imdb': imdb_ratings,
                        'metascore': metascores,
                        'votes': votes})
movie_df['year'] = movie_df['year'].str[-5:-1] 

movie_df

ended


,movie,year,rating,genre,runtime_min,imdb,metascore,votes
0,God Is a Bullet,2023,No rating,"[Action, Crime, Horror]",155.0,NaN,None,NaN
1,Resident Evil: Death Island,2023,R,"[Animation, Action, Adventure]",NaN,NaN,None,NaN
2,The Get Rich Quick Scheme,2023,No rating,[Comedy],NaN,9.1,None,15.0
3,He Killed in Ecstasy,2023,No rating,[Horror],80.0,NaN,None,NaN
4,The Childe,2023,No rating,"[Action, Drama]",118.0,8.6,None,32.0
5,One More Chance,2023,No rating,[Drama],NaN,NaN,None,NaN
6,Gabriel's Redemption: Part One,2023,No rating,[Romance],NaN,NaN,None,NaN
7,El Ruby House,2023,No rating,"[Comedy, Drama]",101.0,NaN,None,NaN
8,Jemputan Ke Neraka,2023,No rating,"[Horror, Mystery, Thriller]",106.0,NaN,None,NaN
9,Un matrimonio mostruoso,2023,No rating,[Comedy],NaN,NaN,None,NaN
